In [1]:
# Install Python dependencies from requirements.txt
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
from pathlib import Path

from network.neural_network import build_model
from dataset.augmentation import RandomAugmentation
from solution.train import Trainer

In [3]:
DATA_PATH = "../data/"

def make_heads():
    head_cls = nn.Sequential(
        nn.Dropout(0.6),
        nn.Linear(256, 135),
    )

    head_cnt = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(128, 6),
    )
    return head_cls, head_cnt


def make_train_transform():
    return RandomAugmentation(
        p_hflip=0.4,
        p_vflip=0.4,
        p_rotate90=0.4,
        p_brightness=0.4,
        p_noise=0.4,
    )


def run_experiment(
    *,
    lambda_cnt: float,
    regression_only: bool,
    max_epochs: int = 100,
    target_accuracy: float = 0.5,
    experiment_name: str = "",
):
    torch.manual_seed(1)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(1)

    head_cls, head_cnt = make_heads()

    config = {
        "lambda_cnt": lambda_cnt,
    }

    model = build_model(
        config=config,
        head_cls=head_cls,
        head_cnt=head_cnt,
    )

    train_transform = make_train_transform()

    trainer = Trainer(
        data_dir=Path(DATA_PATH),
        model=model,
        train_transform=train_transform,
    )

    print(
        f"\n=== Starting experiment: {experiment_name} "
        f"(lambda_cnt={lambda_cnt}, regression_only={regression_only}) ==="
    )

    history = trainer.train(
        epochs=max_epochs,
        verbose=True,
        target_accuracy=target_accuracy,
        regression_only=regression_only,
    )

    trainer.plot_losses(title=f"{experiment_name} – loss")
    trainer.plot_validation_metrics(title=f"{experiment_name} – val metrics")
    trainer.plot_confusion_matrix(
        title=f"{experiment_name} – Val confusion matrix",
        normalize=True,
    )
    trainer.report_best_worst_classes(k=20)

    return trainer, history


def run_classification_only():
    return run_experiment(
        lambda_cnt=0.0,
        regression_only=False,
        max_epochs=100,
        target_accuracy=0.5,
        experiment_name="classification_only",
    )


def run_regression_only():

    return run_experiment(
        lambda_cnt=1.0,
        regression_only=True,
        max_epochs=100,
        target_accuracy=1.0,
        experiment_name="regression_only",
    )


def run_multitask(lambda_cnt: float = 0.3):
    """Eksperyment 3: multitask z wybranym lambda_cnt."""
    return run_experiment(
        lambda_cnt=lambda_cnt,
        regression_only=False,
        max_epochs=100,
        target_accuracy=0.5,
        experiment_name=f"multitask_lambda_{lambda_cnt}",
    )


In [4]:
trainer_reg, hist_reg = run_regression_only()
trainer_cls, hist_cls = run_classification_only()
trainer_multi, hist_multi = run_multitask(lambda_cnt=1.4)


=== Starting experiment: regression_only (lambda_cnt=1.0, regression_only=True) ===


D:\Projekty\sieci_neuronowe\dataset\dataset.py:93: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  arr = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))


Epoch 1: train_loss=1.4154 cls=0.0000 cnt=1.4154
          val_loss=6.2459 acc=0.0110 rmse=2.6897 mae=1.6492


KeyboardInterrupt: 